In [54]:
import numpy as np
import cudf
import cuspatial
from shapely.geometry import Point, Polygon
import geopandas
import time 



N = 10000
lenpoly = 1000
N_polys = 10000
polygon_list = [None]*N_polys
for j in range(N_polys):
    shift = np.random.uniform(-1,1)
    polygon = [[np.sin(x)+shift,np.cos(x)+shift] for x in np.linspace(0,2*np.pi,lenpoly)]
    polygon_arr = np.array(polygon)
    polygon_list[j] = polygon_arr

points = np.random.uniform(-1.5, 1.5, size=(N, 2))

st = time.time()
points_list = [Point(points[i,:]) for i in range(points.shape[0])]
shapekly_result_list = [None]*N_polys
for j in range(N_polys):
    shapely_poly = Polygon(polygon_list[j])
    shapely_result = shapely_poly.contains(points_list)
    shapekly_result_list[j] = shapely_result
end = time.time()

print(end-st)


st = time.time()

shapely_points_list = [Point(points[i,:]) for i in range(points.shape[0])]
shapely_poly_list = [Polygon(polygon) for polygon in polygon_list]

cupoints = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_points_list))
cupoly = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_poly_list))
current_index = 0
next_index = 0
while next_index < N_polys:
    if N_polys - current_index > 30:
        next_index = current_index + 30
    else:
        next_index = N_polys
    result = cuspatial.point_in_polygon(cupoints,
                                    cupoly[current_index:next_index]                   
                                    )
    current_index = next_index
end = time.time()

print(end-st)

st = time.time()
interleaved_points = points.flatten()
cupoints2 = cuspatial.GeoSeries.from_points_xy(interleaved_points)

shapely_poly = [Polygon(polygon) for polygon in polygon_list]
cupoly = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_poly))


current_index = 0
next_index = 0
while next_index < N_polys:
    if N_polys - current_index > 30:
        next_index = current_index + 30
    else:
        next_index = N_polys
    result = cuspatial.point_in_polygon(cupoints2,
                                    cupoly[current_index:next_index]                   
                                    )
    current_index = next_index
    
end = time.time()
print(end-st)


1483.4108567237854
34.84890580177307
34.682761669158936


In [50]:
import numpy as np
import cuspatial
from shapely.geometry import Point, Polygon

N=10

points = np.random.uniform(-1.5, 1.5, size=(N, 2))
shapely_points_list = [Point(points[i,:]) for i in range(points.shape[0])]
cupoints = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_points_list))
print(cupoints)
print(cupoints[0:10])


0    POINT (-0.93174 -0.02952)
1     POINT (-1.06345 1.31086)
2     POINT (1.04204 -0.18691)
3      POINT (1.17295 0.06225)
4      POINT (0.92646 0.68984)
5    POINT (-1.11481 -1.07770)
6      POINT (1.18884 0.36618)
7     POINT (-0.05232 1.07995)
8    POINT (-0.97503 -0.75981)
9      POINT (1.46457 0.38490)
dtype: geometry
0    POINT (-0.93174 -0.02952)
1     POINT (-1.06345 1.31086)
2     POINT (1.04204 -0.18691)
3      POINT (1.17295 0.06225)
4      POINT (0.92646 0.68984)
5    POINT (-1.11481 -1.07770)
6      POINT (1.18884 0.36618)
7     POINT (-0.05232 1.07995)
8    POINT (-0.97503 -0.75981)
9      POINT (1.46457 0.38490)
dtype: geometry


In [34]:
a = []*10
print(a)

[]
